In [1]:
from selenium.common import NoSuchElementException
from selenium import webdriver
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()

# Charger la page Web
driver.get("https://www.rottentomatoes.com/browse/movies_at_home/sort:popular?page=5")
# Vérifier si le bouton "Reject All" existe
try:

    reject_all_button =driver.find_element(By.ID,"onetrust-reject-all-handler")
    # Si le bouton existe, cliquer dessus
    reject_all_button.click()
except NoSuchElementException:
    # Si le bouton n'existe pas, ne rien faire
    pass
for i in range(20):
    try:

        load_more_button = driver.find_element(By.XPATH, "//button[@data-discoverygridsmanager='btnLoadMore']")
        # Si le bouton existe, cliquer dessus
        load_more_button.click()
    except NoSuchElementException:
    # Si le bouton n'existe pas, ne rien faire
        print('The button does not exist')
        pass
    time.sleep(1) # Pause de 1 seconde

# Extraire le contenu HTML de la page Web mise à jour
html = driver.page_source

# Fermer la fenêtre du navigateur
driver.quit()

In [2]:
def get_info(soup):
    list = soup.find("ul", {"id" : "info"})
    film_details = {}
    title = soup.find("h1", {"data-qa" : "score-panel-movie-title"}).text

    film_details["title"] = title



    for child in list.children:
        if child.find("b") != -1 :
            category = child.find("b").text
            category = category.replace(':','')
            value = child.find("span").text
            value = value.strip().replace('\n', '').replace(" ",'').replace(",", ' ')
            film_details[category] = value
    return film_details

In [3]:
soup = BeautifulSoup(html)

In [4]:
film_divs= soup.findAll(class_='js-tile-link')

In [5]:
films_link = []
for film in film_divs:
    film_link = {}
    link_div = (film.find('a', {"data-track" : "scores"}))
    if link_div is not None:
        audiencescore = link_div.find("score-pairs")["audiencescore"]
        tomatoescore = link_div.find("score-pairs")["criticsscore"]

        film_link["link"] = link_div['href']
        film_link["audiencescore"] = audiencescore
        film_link["tomatoescore"]  = tomatoescore
        films_link.append(film_link)

In [ ]:
base_url = 'https://www.rottentomatoes.com'
film_details_list = []
number_film = 0
for film_name in films_link:
    print(number_film, "/", len(films_link))

    url = base_url + film_name["link"]
    print(url)
    response = requests.get(url)
    if response.ok:
        soup = BeautifulSoup(response.content)
        infos = get_info(soup)
        infos["audiencescore"] = film_name["audiencescore"]
        infos["tomatoescore"] = film_name["tomatoescore"]
        film_details_list.append(infos)
    number_film += 1


In [7]:
df = pd.DataFrame(data=film_details_list)
df = df.drop(["View the collection", "Aspect Ratio"], axis=1)

In [8]:
df.columns

Index(['title', 'Rating', 'Genre', 'Original Language', 'Director', 'Producer',
       'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)',
       'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix',
       'audiencescore', 'tomatoescore'],
      dtype='object')

In [9]:
df = df[['title', 'tomatoescore','audiencescore','Rating', 'Genre', 'Original Language', 'Director', 'Producer',
       'Writer', 'Release Date (Theaters)', 'Release Date (Streaming)',
       'Box Office (Gross USA)', 'Runtime', 'Distributor', 'Sound Mix']]

In [10]:
df = df.fillna('')

In [11]:
df['Genre'] = df['Genre'].apply(lambda x: x.split())
df['Director'] = df['Director'].apply(lambda x: x.split())
df['Producer'] = df['Producer'].apply(lambda x: x.split())
df['Writer'] = df['Writer'].apply(lambda x: x.split())


In [14]:
df

,title,tomatoescore,audiencescore,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix
0,Everything Everywhere All at Once,94,86,R(SexualMaterial|Language|SomeViolence),"[Comedy, Adventure, Sci-fi, Fantasy]",English,"[DanKwan, DanielScheinert]","[JoeRusso, AnthonyRusso, MikeLarocca, DanKwan,...","[DanKwan, DanielScheinert]",Apr8 2022 wide,Jun7 2022,$75.7M,2h12m,A24,DolbyDigital
1,Cocaine Bear,68,71,R(LanguageThroughout|DrugContent|BloodyViolenc...,"[Mystery&thriller, Comedy]",English,[ElizabethBanks],"[PhilLord, ChrisMiller, ElizabethBanks, MaxHan...",[JimmyWarden],Feb24 2023 wide,Mar21 2023,$58.6M,1h35m,UniversalPictures,
2,The Whale,64,91,R(SexualContent|Language|SomeDrugUse),[Drama],English,[DarrenAronofsky],[],[SamuelD.Hunter],Dec21 2022 wide,Feb21 2023,$17.2M,1h57m,A24,DolbyDigital
3,Boston Strangler,66,76,R(Language|SomeViolentContent),"[Crime, Drama]",English,[MattRuskin],"[TomAckerley, JoseyMcNamara, MichaelA.Pruss, R...",[MattRuskin],,Mar17 2023,,1h52m,Hulu,DolbyDigital
4,Dragged Across Concrete,76,69,R(Language|GrislyImages|SomeSexuality/Nudity|S...,"[Crime, Drama]",English,[S.CraigZahler],"[KeithKjarval, DallasSonnier, TylerJackson, Ja...",[S.CraigZahler],Mar22 2019 limited,Mar22 2019,,2h39m,SummitEntertainment LionsgateFilms,DolbyDigital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,Pirates of the Caribbean: Dead Men Tell No Tales,30,60,PG-13(SomeSuggestiveContent|AdventureViolence),"[Adventure, Action, Fantasy]",English,"[JoachimRønning, EspenSandberg]",[JerryBruckheimer],[JeffNathanson],May26 2017 wide,Oct3 2017,$172.5M,2h3m,WaltDisney,DolbyAtmos DolbyDigital Datasat
471,Carol,94,74,R(Nudity|BriefLanguage|SceneofSexuality),"[Romance, Drama, Lgbtq+]",English,[ToddHaynes],"[ElizabethKarlsen, StephenWoolley, ChristineVa...",[PhyllisNagy],Jan8 2016 wide,Mar15 2016,$12.7M,1h58m,WeinsteinCo.,
472,The Tomorrow War,52,76,PG-13(Language|IntenseSci-FiViolence|Action|So...,"[Sci-fi, Action, Adventure]",English,[ChrisMcKay],"[DavidEllison, DanaGoldberg, DonGranger, Jules...",[ZachDean],,Jul2 2021,,2h18m,,
473,It Comes at Night,88,44,R(Language|DisturbingImages|Violence),"[Horror, Mystery&thriller]",English,[TreyEdwardShults],"[DavidKaplan, AndreaRoa]",[TreyEdwardShults],Jun9 2017 wide,Sep12 2017,$14.0M,1h37m,A24,


In [15]:
df.to_csv("FilmCSV.csv")